In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.trend import SMAIndicator
from IPython.display import clear_output


payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2000-01-01"
end_date="2021-10-01"
intr="1wk"
form="{:.2%}"
ran=200
for i in range(0,ran):
    try: 
        stk=sip[i]
        df = yf.download(stk, start_date, end_date,interval=intr).dropna()
        df['Close_1']=df['Close'].shift(1)
        df['Volume_1']=df.Volume.shift(1)
        
        df['efi']=ForceIndexIndicator(close=df.Close_1,volume=df.Volume_1,window=14,fillna=True).force_index()
        df['efi_rsi']=RSIIndicator(close=df.efi,window=14,fillna=True).rsi()
        
        df["k"]=StochRSIIndicator(close=df.Close_1,window=14,smooth1=3, smooth2=3).stochrsi_k()
        df["ChngP"]=(df.Close-df.Open)/df.Open
        
        df['sma200']=SMAIndicator(close=df.Close_1,window=200,fillna=True).sma_indicator()
        mxm=np.max(df['Close'])
        
        Condtions = [
        (df['Close']>0)
        &(df["k"]<.10)
        &(df.efi_rsi<40)
        &(df.Close_1<df.sma200)
        ,
        (df['Close']>0)    
        &(df["k"]>.90)
        &(df.efi_rsi>60)
        #&(df.Close_1<df.sma200)
        ]
        
        Outcomes=['ovs','ovb']
        
        df['trade']=np.select(Condtions,Outcomes,default=0)
        
        ovrs=df[df.trade=='ovs']
        ovrb=df[df.trade=='ovb']
        ovrs_chng=np.average(ovrs.ChngP)
        ovrb_chng=np.average(ovrb.ChngP)
        aver_chng=np.average(df.ChngP)
        ovrs_edg=(ovrs_chng-aver_chng)
        ovrb_edg=(ovrb_chng-aver_chng)
        temp=pd.Series([stk,aver_chng,ovrs_edg,ovrb_edg])
        dfout=dfout.append(temp,ignore_index=True)
        clear_output(wait=True)
        print(i ,'/', ran)
    except: 
        pass
dfout.columns=['stk','aver.chng','ovrs_edg','ovrb_edg']
dfout=dfout.dropna()
print(dfout)
print('oversold edge',form.format(np.average(dfout.ovrs_edg)))
print('overbought edge',form.format(np.average(dfout.ovrb_edg)))
#dfout.to_csv('stochrsi_week_edge.csv')

199 / 200
      stk  aver.chng  ovrs_edg  ovrb_edg
0     MMM   0.001650  0.005507 -0.001490
1     ABT   0.002352 -0.000184 -0.002892
2    ABBV   0.002609  0.014278  0.003188
3    ABMD   0.004961  0.026464  0.006416
4     ACN   0.005396  0.008946 -0.053316
..    ...        ...       ...       ...
194    FE   0.000794  0.010393 -0.003792
195   FRC   0.004019 -0.006951  0.006767
196  FISV   0.003549  0.014837  0.023472
198   FMC   0.002475  0.008970 -0.013311
199     F  -0.001614 -0.002278  0.005943

[180 rows x 4 columns]
oversold edge 1.12%
overbought edge -0.20%
